# Setup

In [1]:
import os
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
from scipy.optimize import root, fsolve, newton, brentq, bisect

# line cycler adapted to colourblind people
from cycler import cycler
colourblind_cycler = (cycler(color=['#E69F00', '#56B4E9', '#009E73', '#0072B2', '#D55E00', '#CC79A7', '#F0E442']) +
                      cycler(linestyle=['-', '--', '-.', ':', "-", "--", "-."]))
plt.rc("axes", prop_cycle=colourblind_cycler)

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{newpxtext}\usepackage{newpxmath}\usepackage{commath}\usepackage{mathtools}")
plt.rc("font", family="serif", size=14.)
plt.rc("savefig", dpi=200)
plt.rc("legend", fontsize="medium", fancybox=True, framealpha=0.5)


from nodepy import *

rk4 = rk.loadRKM('RK44').__num__()
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

# http://people.math.sfu.ca/~jverner/RKV87.IIa.Robust.00000754677.081208.FLOAT40OnWeb
a = np.zeros((14, 14))
b = np.zeros(14)

a[2,1] =  .25

a[3,1] =  .8740084650491523205268632759487741197705e-1
a[3,2] =  .2548760493865432175308795062034568513581e-1

a[4,1] =  .4233316929133858267716535433070866141732e-1
a[4,2] =  0.
a[4,3] =  .1269995078740157480314960629921259842520

a[5,1] =  .4260950588874226149488144523757227409094
a[5,2] =  0.
a[5,3] = -1.598795284659152326542773323065718111709
a[5,4] =  1.596700225771729711593958870689995370799

a[6,1] =  .5071933729671392951509061813851363923933e-1
a[6,2] =  0.
a[6,3] =  0.
a[6,4] =  .2543337726460040758275471440887777803137
a[6,5] =  .2039468900572819946573622377727085804470

a[7,1] = -.2900037471752311097038837928542589612409
a[7,2] =  0.
a[7,3] =  0.
a[7,4] =  1.344187391026078988943868110941433700318
a[7,5] = -2.864777943361442730961110382703656282947
a[7,6] =  2.677594299510594851721126064616481543870

a[8,1] =  .9853501133799354646974040298072701428476e-1
a[8,2] =  0.
a[8,3] =  0.
a[8,4] =  0.
a[8,5] =  .2219268063075138484202403649819738790358
a[8,6] = -.1814062291180699431269033828807395245747
a[8,7] =  .1094441147256254823692261491803863125415e-1

a[9,1] =  .3871105254573114467944461816516637340565
a[9,2] =  0.
a[9,3] =  0.
a[9,4] = -1.442445497485527757125674555307792776717
a[9,5] =  2.905398189069950931769134644923384844174
a[9,6] = -1.853771069630105929084333267581197802518
a[9,7] =  .1400364809872815426949732510977124147922
a[9,8] =  .5727394081149581657574677462444770648875

a[10,1] = -.1612440344443930810063001619791348059544
a[10,2] =  0.
a[10,3] =  0.
a[10,4] = -.1733960295735898408357840447396256789490
a[10,5] = -1.301289281406514740601681274517249252974
a[10,6] =  1.137950375173861730855879213143100347212
a[10,7] = -.3174764966396688010692352113804302469898e-1
a[10,8] =  .9335129382493366643981106448605688485659
a[10,9] = -.8378631833473385270330085562961643320150e-1

a[11,1] = -.1919944488158953328151080465148357607314e-1
a[11,2] =  0.
a[11,3] =  0.
a[11,4] =  .2733085726526428490794232625401612427562
a[11,5] = -.6753497320694437291969161121094238085624
a[11,6] =  .3415184981384601607173848997472838271198
a[11,7] = -.6795006480337577247892051619852462939191e-1
a[11,8] =  .9659175224762387888426558649121637650975e-1
a[11,9] =  .1325308251118210118072103846654538995123
a[11,10] =  .3685495936038611344690632995153166681295

a[12,1] =  .6091877403645289867688841211158881778458
a[12,2] =  0.
a[12,3] =  0.
a[12,4] = -2.272569085898001676899980093141308839972
a[12,5] =  4.757898342694029006815525588191478549755
a[12,6] = -5.516106706692758482429468966784424824484
a[12,7] =  .2900596369680119270909581856594617437818
a[12,8] =  .5691423963359036822910985845480184914563
a[12,9] =  .7926795760332167027133991620589332757995
a[12,10] =  .1547372045328882289412619077184989823205
a[12,11] =  1.614970895662181624708321510633454443497

a[13,1] =  .8873576220853471966321169405198102270488
a[13,2] =  0.
a[13,3] =  0.
a[13,4] = -2.975459782108536755851363280470930158198
a[13,5] =  5.600717009488163059799039254835009892383
a[13,6] = -5.915607450536674468001493018994165735184
a[13,7] =  .2202968915613492701687914254080763833124
a[13,8] =  .1015509782446221666614327134090299699755
a[13,9] =  1.151434564738605590978039775212585055356
a[13,10] =  1.929710166527123939613436190080584365307
a[13,11] =  0.
a[13,12] =  0.

b[1] =  .4472956466669571420301584042904938246647e-1
b[2] =  0.
b[3] =  0.
b[4] =  0.
b[5] =  0.
b[6] =  .1569103352770819981336869801072664540918
b[7] =  .1846097340815163774070245187352627789204
b[8] =  .2251638060208699104247941940035072197092
b[9] =  .1479461565197023468700517988544914175374
b[10] =  .7605554244495582526979836191033649101273e-1
b[11] =  .1227729023501861961082434631592143738854
b[12] =  .4181195863899163158338484280087188237679e-1
b[13] =  0.

verner8 = rk.ExplicitRungeKuttaMethod(A=a[1:,1:], b=b[1:]).__num__()

methods = [ssp2, ssp3, ssp104, rk4, bs5, verner8]
method_labels = ["SSPRK(2,2)", "SSPRK(3,3)", "SSPRK(10,4)", "RK(4,4)", "BSRK(8,5)", "Verner(13,8)"]


def compute_rest(rkm, dt, f, eta, deta, w0):
    """Compute the term which is set to zero by relaxed explicit Runge-Kutta methods 
    for general (convex) functionals."""
    s = len(rkm)
    y = np.zeros((s, len(w0))) # stage values
    F = np.zeros((s, len(w0))) # right hand sides
    for i in range(s):
        y[i,:] = w0.copy()
        for j in range(i):
            y[i,:] += rkm.A[i,j]*dt*F[j,:]
        F[i,:] = f(y[i,:])
    
    direction = dt * sum([rkm.b[i]*F[i] for i in range(s)])
    estimate = dt * sum([rkm.b[i]*np.dot(deta(y[i,:]),F[i]) for i in range(s)])
    r = lambda gamma: eta(w0+gamma*direction) - eta(w0) - gamma*estimate
    return r
    

def convex_relaxed_ERK(rkm, dt, f, eta, deta, w0, t_final, 
                       relaxed=True, method="brentq", tol=1.e-14, maxiter=10000, jac=False, newdt=True, 
                       debug=False, correct_last_step=True):
    """Relaxed explicit Runge-Kutta method for convex functionals. It is also possible to apply
    these schemes to general functionals. In that case, some root finding procedure have to be adapted
    slightly."""
    
    w = np.array(w0) # current value of the unknown function
    t = 0 # current time
    ww = np.zeros([np.size(w0), 1]) # values at each time step
    ww[:,0] = w.copy()
    tt = np.zeros(1) # time points for ww
    tt[0] = t
    b = rkm.b
    s = len(rkm)
    y = np.zeros((s, np.size(w0))) # stage values
    F = np.zeros((s, np.size(w0))) # stage derivatives
    max_gammam1 = 0.  # max(gamma-1) over all timesteps
    old_gamma = 1.0
    
    
    # Because of the scaling by gam, the time step which should hit t_final might be a bit too short.
    # In that case, accept this step as the last one in order to terminate the integration.
    while t < t_final and not np.isclose(t, t_final):
        if t + dt > t_final:
            dt = t_final - t
        
        for i in range(s):
            y[i,:] = w.copy()
            for j in range(i):
                y[i,:] += rkm.A[i,j]*dt*F[j,:]
            F[i,:] = f(y[i,:])
        
        if relaxed and ((not np.isclose(dt, t_final - t)) or correct_last_step):
            direction = dt * sum([b[i]*F[i,:] for i in range(s)])
            estimate = dt * sum([b[i]*np.dot(deta(y[i,:]),F[i,:]) for i in range(s)])

            r = lambda gamma: eta(w+gamma*direction) - eta(w) - gamma*estimate
            if debug:
                print('r(1): ', r(1))
            rjac= lambda gamma: np.array([np.dot(deta(w+gamma*direction), direction) - estimate])
            
            if rjac == False:
                use_jac = False
            else:
                use_jac = rjac
            
            if method == "newton":
                gam = newton(r, old_gamma, fprime=rjac, tol=tol, maxiter=maxiter)
                success = True
                msg = "Newton method did not converge"
            elif method == "brentq" or method == "bisect":
                # For convex functionals, additional insights are provided: There is exactly one root
                # and r is negative for smaller gamma and positive for bigger gamma. Thus, we can use
                left= 0.9 * old_gamma
                right = 1.1 * old_gamma
                while r(left) > 0:
                    right = left
                    left *= 0.5
                while r(right) < 0:
                    left = right
                    right *= 2.0
                # For general functionals, we might need to use omething like:
                #left= old_gamma - 0.1
                #right = old_gamma + 0.1
                #while r(left) * r(right) > 0:
                #    left -= 0.1
                #    right += 0.1
                
                if method == "brentq":
                    gam = brentq(r, left, right, xtol=tol, maxiter=maxiter)
                else:
                    gam = bisect(r, left, right, xtol=tol, maxiter=maxiter)
                success = True
                msg = "%s method did not converge"%method
            else:
                sol = root(r, old_gamma, jac=use_jac, method=method, tol=tol, 
                           options={'xtol': tol, 'maxiter': maxiter})
                gam = sol.x; success = sol.success; msg = sol.message
            
            if success == False:
                print('Warning: fsolve did not converge.')
                print(gam)
                print(msg)
                
            if gam <= 0:
                print('Warning: gamma is negative.')

        else:
            gam = 1.
            
        old_gamma = gam
        
        if debug:
            gm1 = np.abs(1.-gam)
            max_gammam1 = max(max_gammam1,gm1)
            if gm1 > 0.5:
                print(gam)
                raise Exception("The time step is probably too large.")
        
        w = w + dt*gam*sum([b[i]*F[i] for i in range(s)])
        if newdt == True:
            t += gam*dt
        else:
            t += dt
        
        tt = np.append(tt, t)
        ww = np.append(ww, np.reshape(w.copy(), (len(w), 1)), axis=1)
        
    if debug:
        print(max_gammam1)
    
    return tt, ww


figure_dir = "./" 

# Harmonic Oscillator with Quartic Entropy/Energy

In [ ]:
def f(w):
    return np.array([-w[1], w[0]])

def eta(w):
    return w[0]*w[0]*w[0]*w[0] + 2 * w[0]*w[0] * w[1]*w[1] + w[1]*w[1]*w[1]*w[1]

def deta(w):
    return np.array([4*w[0]*w[0]*w[0] + 4*w[0]*w[1]*w[1], 4*w[1]*w[1]*w[1] + 4*w[1]*w[0]*w[0]])

def u_analytical(t):
    w0 = np.cos(t)
    w1 = np.sin(t)
    return np.array([w0, w1])

u0 = [1, 0]

dt = 0.9
t_final = 1000

r = compute_rest(ssp3, dt, f, eta, deta, u0)
gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
plt.plot(gamma, rgamma);
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1]);

In [ ]:
%time tt, uu = convex_relaxed_ERK(ssp3, dt, f, eta, deta, u0, t_final, relaxed=True, method="brentq", tol=1.e-15, newdt=True)

plt.close("all");
H = [eta(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);

print("Error at final time: %.3e" % np.linalg.norm(uu[:,-1] - u_analytical(tt[-1])))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(uu[0,:], uu[1,:])
plt.axis('equal')

# Conserved Exponential Entropy

In [ ]:
def f(w):
    return np.array([-np.exp(w[1]), np.exp(w[0])])

def eta(w):
    return np.exp(w[0]) + np.exp(w[1])

def deta(w):
    return np.array([np.exp(w[0]), np.exp(w[1])])

def u_analytical(t):
    sqrte = np.sqrt(np.e)
    tmp = np.exp((np.e + sqrte) * t)
    w0 = np.log(np.e + sqrte*sqrte*sqrte) - np.log(sqrte + tmp)
    w1 = np.log(tmp * (np.e + sqrte) / (sqrte + tmp))
    return np.array([w0, w1])

u0 = [1.0, 0.5]

dt = 0.01
t_final = 5

r = compute_rest(ssp3, dt, f, eta, deta, u0)
gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
plt.plot(gamma, rgamma);
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1]);

In [ ]:
%time tt, uu = convex_relaxed_ERK(bs5, dt*1.e-1, f, eta, deta, u0, t_final, relaxed=True, tol=1.e-14, method="brentq", newdt=True)

plt.close("all");
H = [eta(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);

plt.figure()
err = [np.linalg.norm(uu[:,i] - u_analytical(tt[i])) for i in range(uu.shape[1])]
plt.plot(tt[1:], err[1:]);
plt.xscale("log")
plt.yscale("log")
plt.xlabel("$t$"); plt.ylabel("Error");

print("Error at final time: %.3e" % np.linalg.norm(uu[:,-1] - u_analytical(tt[-1])))

## Plots Used in the Paper

### $r(\gamma)$ for SSPRK(3,3)

In [ ]:
plt.figure(figsize=(4,4))

for dt in [0.12, 0.1,0.08]:
    r = compute_rest(ssp3, dt, f, eta, deta, u0)
    gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
    plt.plot(gamma, rgamma, label="$\Delta t = %.2f$"%dt);
    
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1])
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "cons_exp_entropy__r_over_gamma.pdf"), bbox_inches="tight")

### $r(\gamma=1)$ for Several Methods

In [ ]:
%%time

dts = np.logspace(-3, 0.5, 100)

plt.figure(figsize=(6,5))
for method_idx in range(len(methods)):
    r_at_one = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        r = compute_rest(methods[method_idx], dts[dt_idx], f, eta, deta, u0)
        r_at_one[dt_idx] = np.abs(r(1.))
    plt.plot(dts, r_at_one, label=method_labels[method_idx]);
    
order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 1.e1 * order_dts**3, "-o", color="gray", label="$\mathcal{O}(\Delta t^3)$")
plt.plot(order_dts, .2e1 * order_dts**4, "-P", color="gray", label="$\mathcal{O}(\Delta t^4)$")
plt.plot(order_dts, 8.e0 * order_dts**5, "-X", color="gray", label="$\mathcal{O}(\Delta t^5)$")
order_dts = np.array([dts[int(len(dts)/4)], dts[int(3*len(dts)/5)]])
plt.plot(order_dts, 1.e-2 * order_dts**6, "-s", color="gray", label="$\mathcal{O}(\Delta t^6)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(4*len(dts)/5)]])
plt.plot(order_dts, 1.e-4 * order_dts**9, "-D", color="gray", label="$\mathcal{O}(\Delta t^9)$")

plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("$\Delta t$"); plt.ylabel("$| r(\gamma=1) |$")
plt.xscale("log"); plt.yscale("log")
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "cons_exp_entropy__r_at_one.pdf"), bbox_inches="tight")

### Convergence

In [ ]:
%%time

dts = np.logspace(-2.5, 0., 100)

ylim = (1.e-15, 1.e0)

plt.figure(figsize=(3, 4.5))
for method_idx in range(len(methods)):
    errors = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        tt, uu = convex_relaxed_ERK(methods[method_idx], dts[dt_idx], f, eta, deta, u0, t_final, 
                                    relaxed=False, method="brentq", newdt=True, correct_last_step=False)
        errors[dt_idx] = np.linalg.norm(uu[:,-1] - u_analytical(tt[-1]))
    plt.plot(dts, errors, label=method_labels[method_idx]);

order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 5.e1 * order_dts**2, "-o", color="gray", label="$\mathcal{O}(\Delta t^2)$")
plt.plot(order_dts, 1.e2 * order_dts**3, "-P", color="gray", label="$\mathcal{O}(\Delta t^3)$")
plt.plot(order_dts, 8.e0 * order_dts**4, "-X", color="gray", label="$\mathcal{O}(\Delta t^4)$")
order_dts = np.array([dts[int(len(dts)/4)], dts[int(3*len(dts)/5)]])
plt.plot(order_dts, 1.e-2 * order_dts**5, "-s", color="gray", label="$\mathcal{O}(\Delta t^5)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(4*len(dts)/5)]])
plt.plot(order_dts, 1.e-4 * order_dts**8, "-D", color="gray", label="$\mathcal{O}(\Delta t^8)$")

plt.xlabel("$\Delta t$"); plt.ylabel("Error at $t = %.0f$"%t_final)
plt.xscale("log"); plt.yscale("log")
plt.ylim(ylim)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "cons_exp_entropy__convergence_standard.pdf"), bbox_inches="tight")

ax = plt.gca()
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=6)
plt.savefig(os.path.join(figure_dir, "cons_exp_entropy__convergence_legend.pdf"), bbox_inches="tight")

In [ ]:
%%time

plt.figure(figsize=(3, 4.5))
for method_idx in range(len(methods)):
    errors = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        tt, uu = convex_relaxed_ERK(methods[method_idx], dts[dt_idx], f, eta, deta, u0, t_final, 
                                    relaxed=True, method="brentq", tol=1.e-14, newdt=True, correct_last_step=False)
        errors[dt_idx] = np.linalg.norm(uu[:,-1] - u_analytical(tt[-1]))
    plt.plot(dts, errors, label=method_labels[method_idx]);

order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 2.e1 * order_dts**2, "-o", color="gray", label="$\mathcal{O}(\Delta t^2)$")
plt.plot(order_dts, 1.e1 * order_dts**3, "-P", color="gray", label="$\mathcal{O}(\Delta t^3)$")
plt.plot(order_dts, 8.e0 * order_dts**4, "-X", color="gray", label="$\mathcal{O}(\Delta t^4)$")
order_dts = np.array([dts[int(len(dts)/4)], dts[int(3*len(dts)/5)]])
plt.plot(order_dts, 1.e-2 * order_dts**5, "-s", color="gray", label="$\mathcal{O}(\Delta t^5)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(4*len(dts)/5)]])
plt.plot(order_dts, 1.e-4 * order_dts**8, "-D", color="gray", label="$\mathcal{O}(\Delta t^8)$")

plt.xlabel("$\Delta t$"); plt.ylabel("Error at $t = %.0f$"%t_final)
plt.xscale("log"); plt.yscale("log")
plt.ylim(ylim)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "cons_exp_entropy__convergence_relaxed.pdf"), bbox_inches="tight")

In [ ]:
%%time

plt.figure(figsize=(3, 4.5))
for method_idx in range(len(methods)):
    errors = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        tt, uu = convex_relaxed_ERK(methods[method_idx], dts[dt_idx], f, eta, deta, u0, t_final, 
                                    relaxed=True, method="brentq", tol=1.e-14, newdt=False, correct_last_step=False)
        errors[dt_idx] = np.linalg.norm(uu[:,-1] - u_analytical(tt[-1]))
    plt.plot(dts, errors, label=method_labels[method_idx]);

order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 5.e0 * order_dts**2, "-o", color="gray", label="$\mathcal{O}(\Delta t^2)$")
order_dts = np.array([dts[int(len(dts)/3)], dts[int(4*len(dts)/5)]])
plt.plot(order_dts, 1.e0 * order_dts**3, "-P", color="gray", label="$\mathcal{O}(\Delta t^3)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(4*len(dts)/5)]])
plt.plot(order_dts, 1.e-2 * order_dts**4, "-X", color="gray", label="$\mathcal{O}(\Delta t^4)$")
#order_dts = np.array([dts[int(len(dts)/4)], dts[int(3*len(dts)/5)]])
#plt.plot(order_dts, 1.e-2 * order_dts**5, "-s", color="gray", label="$\mathcal{O}(\Delta t^5)$")
#order_dts = np.array([dts[int(len(dts)/2)], dts[int(4*len(dts)/5)]])
#plt.plot(order_dts, 1.e-4 * order_dts**8, "-D", color="gray", label="$\mathcal{O}(\Delta t^8)$")

plt.xlabel("$\Delta t$"); plt.ylabel("Error at $t = %.0f$"%t_final)
plt.xscale("log"); plt.yscale("log")
plt.ylim(ylim)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "cons_exp_entropy__convergence_relaxed_olddt.pdf"), bbox_inches="tight")

# Dissipated Exponential Entropy

In [ ]:
def f(w):
    return np.array([-np.exp(w[0]), -np.exp(w[1])])

def eta(w):
    return np.exp(w[0]) + np.exp(w[1])

def deta(w):
    return np.array([np.exp(w[0]), np.exp(w[1])])

def u_analytical(t):
    w0 = -np.log(1 / np.e + t)
    w1 = -np.log(1/np.sqrt(np.e) + t)
    return np.array([w0, w1])

u0 = [1.0, 0.5]

dt = 0.1
t_final = 5

r = compute_rest(ssp3, dt, f, eta, deta, u0)
gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
plt.plot(gamma, rgamma);
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1]);

In [ ]:
%time tt, uu = convex_relaxed_ERK(ssp3, dt, f, eta, deta, u0, t_final, relaxed=True, method="brentq", newdt=True)

plt.close("all");
H = [eta(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);

print("Error at final time: %.3e" % np.linalg.norm(uu[:,-1] - u_analytical(tt[-1])))

### $r(\gamma)$ for SSPRK(3,3)

In [ ]:
plt.figure(figsize=(4,4))

for dt in [0.12, 0.1,0.08]:
    r = compute_rest(ssp3, dt, f, eta, deta, u0)
    gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
    plt.plot(gamma, rgamma, label="$\Delta t = %.2f$"%dt);
    
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1])
plt.legend()
plt.tight_layout()
#plt.savefig(os.path.join(figure_dir, "diss_exp_entropy__r_over_gamma.pdf"), bbox_inches="tight")

### $r(\gamma=1)$ for Several Methods

In [ ]:
%%time

dts = np.logspace(-3, 0.5, 100)

plt.figure(figsize=(6,5))
for method_idx in range(len(methods)):
    r_at_one = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        r = compute_rest(methods[method_idx], dts[dt_idx], f, eta, deta, u0)
        r_at_one[dt_idx] = np.abs(r(1.))
    plt.plot(dts, r_at_one, label=method_labels[method_idx]);
    
order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 1.e1 * order_dts**3, "-o", color="gray", label="$\mathcal{O}(\Delta t^3)$")
plt.plot(order_dts, .2e1 * order_dts**4, "-P", color="gray", label="$\mathcal{O}(\Delta t^4)$")
plt.plot(order_dts, 8.e0 * order_dts**5, "-X", color="gray", label="$\mathcal{O}(\Delta t^5)$")
order_dts = np.array([dts[int(len(dts)/4)], dts[int(3*len(dts)/5)]])
plt.plot(order_dts, 1.e-2 * order_dts**6, "-s", color="gray", label="$\mathcal{O}(\Delta t^6)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(4*len(dts)/5)]])
plt.plot(order_dts, 1.e-4 * order_dts**9, "-D", color="gray", label="$\mathcal{O}(\Delta t^9)$")

plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("$\Delta t$"); plt.ylabel("$| r(\gamma=1) |$")
plt.xscale("log"); plt.yscale("log")
plt.tight_layout()
#plt.savefig(os.path.join(figure_dir, "diss_exp_entropy__r_at_one.pdf"), bbox_inches="tight")

## Plots Used in the Paper

### Convergence

In [ ]:
%%time

dts = np.logspace(-2.5, 0., 100)

ylim = (1.e-15, 1.e0)

plt.figure(figsize=(3, 4.5))
for method_idx in range(len(methods)):
    errors = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        tt, uu = convex_relaxed_ERK(methods[method_idx], dts[dt_idx], f, eta, deta, u0, t_final, 
                                    relaxed=False, method="brentq", tol=1.e-14, newdt=True, correct_last_step=False)
        errors[dt_idx] = np.linalg.norm(uu[:,-1] - u_analytical(tt[-1]))
    plt.plot(dts, errors, label=method_labels[method_idx]);

order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 5.e-2 * order_dts**2, "-o", color="gray", label="$\mathcal{O}(\Delta t^2)$")
plt.plot(order_dts, 2.e-1 * order_dts**3, "-P", color="gray", label="$\mathcal{O}(\Delta t^3)$")
plt.plot(order_dts, 8.e-3 * order_dts**4, "-X", color="gray", label="$\mathcal{O}(\Delta t^4)$")
order_dts = np.array([dts[int(len(dts)/3)], dts[int(5*len(dts)/7)]])
plt.plot(order_dts, 1.e-5 * order_dts**5, "-s", color="gray", label="$\mathcal{O}(\Delta t^5)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(5*len(dts)/7)]])
plt.plot(order_dts, 1.e-5 * order_dts**8, "-D", color="gray", label="$\mathcal{O}(\Delta t^8)$")

plt.xlabel("$\Delta t$"); plt.ylabel("Error at $t = %.0f$"%t_final)
plt.xscale("log"); plt.yscale("log")
plt.ylim(ylim)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "diss_exp_entropy__convergence_standard.pdf"), bbox_inches="tight")

ax = plt.gca()
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=6)
plt.savefig(os.path.join(figure_dir, "diss_exp_entropy__convergence_legend.pdf"), bbox_inches="tight")

In [ ]:
%%time

plt.figure(figsize=(3, 4.5))
for method_idx in range(len(methods)):
    errors = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        tt, uu = convex_relaxed_ERK(methods[method_idx], dts[dt_idx], f, eta, deta, u0, t_final, 
                                    relaxed=True, method="brentq", tol=1.e-14, newdt=True, correct_last_step=False)
        errors[dt_idx] = np.linalg.norm(uu[:,-1] - u_analytical(tt[-1]))
    plt.plot(dts, errors, label=method_labels[method_idx]);

order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 7.e-2 * order_dts**2, "-o", color="gray", label="$\mathcal{O}(\Delta t^2)$")
plt.plot(order_dts, 2.e-1 * order_dts**3, "-P", color="gray", label="$\mathcal{O}(\Delta t^3)$")
plt.plot(order_dts, 8.e-3 * order_dts**4, "-X", color="gray", label="$\mathcal{O}(\Delta t^4)$")
order_dts = np.array([dts[int(len(dts)/4)], dts[int(4*len(dts)/7)]])
plt.plot(order_dts, 1.e-4 * order_dts**5, "-s", color="gray", label="$\mathcal{O}(\Delta t^5)$")
order_dts = np.array([dts[int(len(dts)/2)], dts[int(5*len(dts)/7)]])
plt.plot(order_dts, 1.e-5 * order_dts**8, "-D", color="gray", label="$\mathcal{O}(\Delta t^8)$")

plt.xlabel("$\Delta t$"); plt.ylabel("Error at $t = %.0f$"%t_final)
plt.xscale("log"); plt.yscale("log")
plt.ylim(ylim)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "diss_exp_entropy__convergence_relaxed.pdf"), bbox_inches="tight")

In [ ]:
%%time

plt.figure(figsize=(3, 4.5))
for method_idx in range(len(methods)):
    errors = np.zeros_like(dts)
    for dt_idx in range(len(dts)):
        tt, uu = convex_relaxed_ERK(methods[method_idx], dts[dt_idx], f, eta, deta, u0, t_final, 
                                    relaxed=True, method="brentq", tol=1.e-14, newdt=False, correct_last_step=False)
        errors[dt_idx] = np.linalg.norm(uu[:,-1] - u_analytical(tt[-1]))
    plt.plot(dts, errors, label=method_labels[method_idx]);

order_dts = np.array([dts[0], dts[int(len(dts)/2)]])
plt.plot(order_dts, 1.e-1 * order_dts**2, "-o", color="gray", label="$\mathcal{O}(\Delta t^2)$")
plt.plot(order_dts, 2.e-1 * order_dts**3, "-P", color="gray", label="$\mathcal{O}(\Delta t^3)$")
order_dts = np.array([dts[int(len(dts)/6)], dts[int(4*len(dts)/7)]])
plt.plot(order_dts, 1.e-3 * order_dts**4, "-X", color="gray", label="$\mathcal{O}(\Delta t^4)$")

plt.xlabel("$\Delta t$"); plt.ylabel("Error at $t = %.0f$"%t_final)
plt.xscale("log"); plt.yscale("log")
plt.ylim(ylim)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "diss_exp_entropy__convergence_relaxed_olddt.pdf"), bbox_inches="tight")

# Nonlinear Pendulum

In [ ]:
def f(w):
    return np.array([-np.sin(w[1]), w[0]])

def eta(w):
    return 0.5*w[0]**2 - np.cos(w[1])

def deta(w):
    return np.array([w[0], np.sin(w[1])])

u0 = [1.5, 0]

dt = 0.9
t_final = 1000

r = compute_rest(ssp3, dt, f, eta, deta, u0)
gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
plt.plot(gamma, rgamma);
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1]);

In [ ]:
%time tt, uu = convex_relaxed_ERK(ssp3, dt, f, eta, deta, u0, t_final, relaxed=True, method="brentq", tol=1.e-15, newdt=True)

plt.close("all");
H = [eta(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(uu[0,:], uu[1,:])
plt.axis('equal')

## Plots Used in the Paper

## Energy Evolution

In [ ]:
%%time

plt.figure(figsize=(4, 3))
for method_idx in range(len(methods)):
    tt, uu = convex_relaxed_ERK(methods[method_idx], dt, f, eta, deta, u0, t_final, 
                                relaxed=False, method="brentq", tol=1.e-15, newdt=True)
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label=method_labels[method_idx]);

plt.xlabel("$t$"); plt.ylabel("$\eta(u(t)) - \eta(u^0)$"); plt.xlim(tt[0], tt[-1]);
plt.yscale("symlog", linthreshy=1.e-5)
#plt.locator_params(axis='y', numticks=6)
plt.yticks([-1.e1, -1.e-2, -1.e-5, 1.e-5, 1.e-2, 1.e1])
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "pendulum__energy_standard.pdf"), bbox_inches="tight")

ax = plt.gca()
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=6)
plt.savefig(os.path.join(figure_dir, "pendulum__energy_legend.pdf"), bbox_inches="tight")

In [ ]:
%%time

plt.figure(figsize=(4, 3))
for method_idx in range(len(methods)):
    tt, uu = convex_relaxed_ERK(methods[method_idx], dt, f, eta, deta, u0, t_final, 
                                relaxed=True, method="brentq", tol=1.e-15, newdt=True)
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label=method_labels[method_idx]);

plt.xlabel("$t$"); plt.ylabel("$\eta(u(t)) - \eta(u^0)$"); plt.xlim(tt[0], tt[-1]);
plt.yscale("symlog", linthreshy=1.e-18)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "pendulum__energy_relaxed.pdf"), bbox_inches="tight")

In [ ]:
%%time

plt.figure(figsize=(4, 3))
for method_idx in range(len(methods)):
    tt, uu = convex_relaxed_ERK(methods[method_idx], dt, f, eta, deta, u0, t_final, 
                                relaxed=True, method="brentq", tol=1.e-15, newdt=False)
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label=method_labels[method_idx]);

plt.xlabel("$t$"); plt.ylabel("$\eta(u(t)) - \eta(u^0)$"); plt.xlim(tt[0], tt[-1]);
plt.yscale("symlog", linthreshy=1.e-18)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "pendulum__energy_relaxed_olddt.pdf"), bbox_inches="tight")

### Phase Space and Energy

In [ ]:
%%time

fig_energy = plt.figure("energy", figsize=(4, 4))
fig_phase = plt.figure("phase", figsize=(4, 4))

for method_idx in [1,3]:
    for relaxed in [False, True]:
        tt, uu = convex_relaxed_ERK(methods[method_idx], dt, f, eta, deta, u0, t_final, 
                                    relaxed=relaxed, method="brentq", tol=1.e-15, newdt=True)

        plt.figure("energy")
        H = [eta(uu[:,i]) for i in range(uu.shape[1])]
        label = method_labels[method_idx]
        if relaxed:
            label = "%s, relaxed"%label
        plt.plot(tt, H - H[0], label=label);

        plt.figure("phase")
        plt.plot(uu[0,:], uu[1,:], label=label);

plt.figure("energy")
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);
plt.savefig(os.path.join(figure_dir, "pendulum__phase_energy.pdf"), bbox_inches="tight")

ax = plt.gca()
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=6)
plt.savefig(os.path.join(figure_dir, "pendulum__phase_legend.pdf"), bbox_inches="tight")

plt.figure("phase")
plt.axis('equal')
plt.xlabel("$u_1$"); plt.ylabel("$u_2$");
amplitude = 2.0; plt.xlim((-amplitude, amplitude)); plt.ylim((-amplitude, amplitude));
plt.savefig(os.path.join(figure_dir, "pendulum__phase_plot.pdf"), bbox_inches="tight")

# Lotka-Volterra Equations

In [ ]:
a = 1.0
b = 1.0
c = 1.0
d = 1.0

def f(w):
    return np.array([w[0]*(a-b*w[1]), w[1]*(c*w[0]-d)])

def eta(w):
    return c*w[0] - d*np.log(w[0]) + b*w[1] - a*np.log(w[1])

def deta(w):
    return np.array([c - d/w[0], b - a/w[1]])

u0 = [1., 2.]

dt = 0.5
t_final = 10000

r = compute_rest(ssp3, dt, f, eta, deta, u0)
gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
plt.plot(gamma, rgamma);
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1]);

In [ ]:
%time tt, uu = convex_relaxed_ERK(rk4, dt, f, eta, deta, u0, t_final, relaxed=True, method="brentq", newdt=True)

plt.close("all");
H = [eta(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);

In [ ]:
plt.figure(figsize=(4,4))
plt.plot(uu[0,:], uu[1,:])
plt.axis('equal');

# Burgers' Equation with Logarithmic Entropy

In [ ]:
N = 100
cfl = 0.1
mu = 0. # dissipation coefficient

x, dx = np.linspace(-1, 1, N, endpoint=False, retstep=True)
x = x + 0.5*dx # x are the mean values of each cell
u0 = np.exp(-30*x**2)

def f(u):
    # left and right values
    ul = np.zeros(np.size(u)+1)
    ul[1:] = u; ul[1] = u[-1]
    ur = np.zeros_like(ul)
    ur[:-1] = u; ur[-1] = ul[0]
    # compute numerical fluxes
    fluxes = 0.5 * ul * ur - mu*(ur-ul)
    # compute flux differences
    fluxdiff = -(fluxes[1:] - fluxes[:-1]) / dx
    return fluxdiff

def eta(u):
    return sum(-np.log(u)) * dx

def deta(u):
    return -1./u * dx

dt = cfl*dx
t_final = 0.1

r = compute_rest(ssp3, dt, f, eta, deta, u0)
gamma = np.linspace(-0.1, 1.2); rgamma = np.array([r(gam) for gam in gamma])
plt.plot(gamma, rgamma);
plt.xlabel("$\gamma$"); plt.ylabel("$r(\gamma)$"); plt.xlim(gamma[0], gamma[-1]);

In [ ]:
%time tt, uu = convex_relaxed_ERK(rk4, dt, f, eta, deta, u0, t_final, relaxed=True, method="brentq", newdt=True)

plt.close("all");
plt.plot(x, uu[:,-1]);
plt.xlabel("$x$"); plt.ylabel("$u(T, \cdot)$"); plt.xlim(x[0]-0.5*dx, x[-1]+0.5*dx);
plt.figure()
H = [eta(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(tt[0], tt[-1]);